In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [34]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils

import keras

In [14]:
training = pd.read_csv("../src/Data/train.csv")
testing = pd.read_csv("../src/Data/test.csv")

##Separating into Training, and Validation data

In [37]:
seed = 123
np.random.seed(123)

In [20]:
X = training.values[:, 1:]
Y = training.values[:, 0]

In [26]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.25, random_state = 123)

In [10]:
training.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Neural Network Implementation

In [31]:
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y = encoder.transform(Y_train)
dummy_Y_train = np_utils.to_categorical(encoded_Y)

In [32]:
encoder = LabelEncoder()
encoder.fit(Y_valid)
encoded_Y = encoder.transform(Y_valid)
dummy_Y_valid = np_utils.to_categorical(encoded_Y)

In [42]:
def deep_forward_model(nodes = 32,
                       rate = 0.5):
    
    #Building Model
    nn_model = Sequential()
    nn_model.add(Dense(nodes, activation = 'relu', input_dim = 784))
    nn_model.add(Dense(nodes, activation = 'relu'))
    nn_model.add(Dropout(rate = rate, seed = 123))
    nn_model.add(Dense(nodes, activation = 'relu'))
    nn_model.add(Dense(10, activation = 'softmax'))
    
    #Compliling Model
    nn_model.compile(loss = 'categorical_crossentropy', 
                    optimizer = 'adam',
                    metrics = ['accuracy'])
    
    return nn_model

In [ ]:
def nn_tune(model, params, n = 5):
    nn_search = GridSearchCV(estimator=model, param_grid=params, cv = n)
    nn_result = nn_search.fit(x_train_vals, y_train_vals)
    return nn_result.best_params_

In [53]:
model = KerasClassifier(build_fn = deep_forward_model, epochs = 100, batch_size = 5, verbose = 0)

In [54]:
model.fit(X_train, dummy_Y_train)

In [56]:
model.score(X_train, dummy_Y_train)

0.817015882380425

##One way to Evaluate

In [45]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = seed)

In [46]:
cv_results = cross_val_score(model, X_train, dummy_Y_train, cv = kfold)
print("Baseline: {}% ({}%)").format(cv_results.mean()*100, cv_results.std()*100)

KeyboardInterrupt: 